# Проект по SQL

### Цель исследования
Первая задача как аналитика — проанализировать базу данных.
Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Общие данные:
База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские
обзоры книг. 

### Описание таблиц:
##### books cодержит данные о книгах:   
 - book_id — идентификатор книги;   
 - author_id — идентификатор автора;   
 - title — название книги;   
 - num_pages — количество страниц;   
 - publication_date — дата публикации книги;   
 - publisher_id — идентификатор издателя.   

##### authors cодержит данные об авторах:   
 - author_id — идентификатор автора;   
 - author — имя автора    

##### publishers cодержит данные об издательствах:    
 - publisher_id — идентификатор издательства;   
 - publisher — название издательства;   

##### ratings cодержит данные о пользовательских оценках книг:   
 - rating_id — идентификатор оценки;   
 - book_id — идентификатор книги;   
 - username — имя пользователя, оставившего оценку;   
 - rating — оценка книги.   

##### reviews cодержит данные о пользовательских обзорах на книги:   
 - review_id — идентификатор обзора;   
 - book_id — идентификатор книги;   
 - username — имя пользователя, написавшего обзор;   
 - text — текст обзора.   

### Подключение к БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
import warnings; warnings.filterwarnings(action='once')

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

### Знакомство с таблицами

#### Таблица books

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


#### Таблица authors

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Таблица publishers

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


#### Таблица ratings

In [6]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### Таблица reviews

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Задачи

#### 1. Посчитайте, сколько книг вышло после 1 января 2000 года :

In [8]:
query = '''
        SELECT COUNT(book_id) AS КнигСНачала2000года
        FROM books
        WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = con)

,КнигСНачала2000года
0,819


Начиная с 1 января 2000 года в каталог сервиса было добавлено 819 книг.

#### 2. Для каждой книги посчитайте количество обзоров и среднюю оценку :

In [9]:
query = '''
       SELECT b.book_id, 
              b.title AS Заголовок, 
              COUNT(DISTINCT rv.review_id) AS Обзоров, 
              ROUND(AVG(r.rating),3) AS СреднийРейтинг
       FROM books AS b
       LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
       LEFT JOIN  ratings AS r ON b.book_id = r.book_id
       GROUP BY b.book_id, b.title
       ORDER BY Обзоров DESC
'''

pd.io.sql.read_sql(query, con = con)

,book_id,Заголовок,Обзоров,СреднийРейтинг
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


По данным можно сказать что : в каталоге 1000 книг. Минимальное количество обзоров от 0 до 7. Оценки рейтинга судя по данным не слишком зависят от количества обзоров (по количеству не сказать о качестве этих обзоров). Средний рейтиинг всех книг примерно 4.

#### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query = '''
        WITH w AS (
        SELECT publisher_id, book_id
        FROM books
        WHERE num_pages > 50
        )
        SELECT p.publisher AS Издательство
        FROM w 
        INNER JOIN publishers AS p ON w.publisher_id = p.publisher_id
        GROUP BY p.publisher
        ORDER BY COUNT(w.book_id) DESC
        LIMIT 1
'''
pd.io.sql.read_sql(query, con = con)

,Издательство
0,Penguin Books


Большее количество представленных книг с тольшиной более чем 50 страниц находятся у издательства Penguin Books.

#### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [17]:
query = '''
       WITH w AS (
       SELECT book_id
       FROM ratings
       GROUP BY book_id
       HAVING COUNT(rating_id) >= 50
       )
       SELECT a.author AS Автор,
              AVG(r.rating) AS СреднийРейтинг
       FROM books AS b
       INNER JOIN w ON b.book_id = w.book_id
       INNER JOIN authors AS a ON b.author_id = a.author_id
       INNER JOIN ratings AS r ON w.book_id = r.book_id
       GROUP BY a.author_id, a.author
       ORDER BY AVG(r.rating) DESC
       LIMIT 1
'''
pd.io.sql.read_sql(query, con = con)

,Автор,СреднийРейтинг
0,J.K. Rowling/Mary GrandPré,4.287097


Самые популярный авторы с средним рейтингом 4.29 оказались Джоан Роулинг и Мари Грандпре.

#### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
        SELECT COUNT(DISTINCT rv.text)/ COUNT(DISTINCT rv.username) AS avg_rv_count
        FROM ratings AS r
        INNER JOIN reviews AS rv ON r.username = rv.username
        WHERE r.username in (SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 48)
'''
pd.io.sql.read_sql(query, con = con)

,avg_rv_count
0,24


Среднее число обзоров является 24, среди тех пользователей которые поставивили в среднем болеее 48 оценок.

Общий вывод: Магазин книг должен сосредоточиться на расширении ассортимента, например книг издательства Penguin Books, и активно продвигать популярные книги, а так же авторов похожих на Джоан Роулинг и Мари Грандпре. Также следует поощрять и привлекать пользователей, которые активно оценивают книги.